In [1]:
import numpy as np
np.random.seed(1337)
import json, re, nltk, string
from nltk.corpus import wordnet
from gensim.models import Word2Vec
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge
from keras.optimizers import RMSprop
from keras.utils import np_utils
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import pandas as pd

In [3]:
all_bugs_json = 'D:\\BugTriage\\Chrome\\all_data.json'
closed_bugs_json = 'D:\\BugTriage\\Chrome\\classifier_data_0.json'

In [4]:
with open(all_bugs_json) as data_file:
    data = json.load(data_file, strict=False)


In [5]:
data_id= list()
data_issueId = list()
data_issueTitle = list()
data_reportedTime = list()
data_owner = list()
data_description = list()

data_des = []

for i in range(len(data)):
    data_id.append(data[i]['id'])
    data_issueId.append(data[i]['issue_id'])
    data_issueTitle.append(data[i]['issue_title'])
    data_reportedTime.append(data[i]['reported_time'])
    data_owner.append(data[i]['owner'])
    data_des.append(data[i]['description'])

for item in data_des:
    current_desc = item.replace('\r', ' ')
    current_desc = current_desc.replace('\n', ' ')    
    current_desc = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', current_desc)
    current_desc = current_desc[:start_loc]  
    current_desc = re.sub(r'(\w+)0x\w+', '', current_desc)
    current_desc = current_desc.lower()
    current_desc_tokens = nltk.word_tokenize(current_desc)
    current_desc_filter = [word.strip(string.punctuation) for word in current_desc_tokens]

    data_description.append(current_desc)

In [6]:
for item in data:
    current_title = item['issue_title'].replace('\r', ' ')
    current_desc = item['description'].replace('\r', ' ')
    current_desc = current_desc.replace('\n', ' ')

    current_desc = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', current_desc)

    start_loc = current_desc.find("Stack trace:")
    current_desc = current_desc[:start_loc]
    

    current_desc = re.sub(r'(\w+)0x\w+', '', current_desc)
    current_title= re.sub(r'(\w+)0x\w+', '', current_title)

    current_desc = current_desc.lower()
    current_title = current_title.lower()
    #print(current_title)

    current_desc = current_title + current_desc
    #print(current_desc)
    '''current_desc_tokens = nltk.word_tokenize(current_desc)
    current_title_tokens = nltk.word_tokenize(current_title)
    print(current_desc_tokens)
    current_desc_filter = [word.strip(string.punctuation) for word in current_desc_tokens]
    current_title_filter = [word.strip(string.punctuation) for word in current_title_tokens]      

    current_data = current_title_filter + current_desc_filter
    current_data = filter(None, current_data)'''

    data_description.append(current_desc)

In [16]:
deepTriage = pd.DataFrame(data = {'id':data_id, 'issue_id':data_issueId, 'issue_title':data_issueTitle, 'report_time':data_reportedTime, 'owner':data_owner, 'description':data_description})

In [18]:
deepTriage

,id,issue_id,issue_title,report_time,owner,description
0,1,2,Testing if chromium id works,2008-08-30 16:00:21,,testing if chromium id works what steps will r...
1,2,3,This is a test,2008-08-31 02:47:11,,this is a test product version : 0.2.149....
2,3,4,"Scrolling with some scroll mice (touchpad, etc...",2008-09-02 19:02:31,amit@chromium.org,"scrolling with some scroll mice (touchpad, etc..."
3,4,5,Java not working yet,2008-09-02 19:04:27,,java not working yet product version : 0....
4,5,6,Chrome treats links in frames differently than...,2008-09-02 19:08:01,,chrome treats links in frames differently than...
...,...,...,...,...,...,...
383099,383100,349995,Windows GPU bots failing on multiple tests,2014-03-06 18:42:28,ernstm@chromium.org,windows gpu bots failing on multiple tests all...
383100,383101,349996,Pop-up window from inside Flash Player not wor...,2014-03-06 18:42:58,,pop-up window from inside flash player not wor...
383101,383102,349997,URL blacklist policy can't filter by query par...,2014-03-06 18:43:00,kaliamoorthi@chromium.org,url blacklist policy can't filter by query par...
383102,383103,349998,CSS auto-complete suggestion list is incomplet...,2014-03-06 18:43:04,apavlov@chromium.org,css auto-complete suggestion list is incomplet...


In [26]:
len(deepTriage['owner'])

383104

In [ ]:
for i in range(len(deepTriage)):
    if deepTriage['owner'][i] != '':
        print(deepTriage['owner'][i])

In [ ]:
data_owner

In [30]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug

import csv

In [31]:
def deepTriage_data(dataset):

    aug_range = len(dataset)
    f = open('D:/DeepTriage/Chrome_report.csv', 'w', newline='', encoding='utf-8')
    wr = csv.writer(f)
    wr.writerow(dataset)
    for x in range(len(dataset)):
        wr.writerow([dataset['text'][x], dataset['labels'][x]])

    dataset.to_csv
    
    f.close()

In [35]:
deepTriage.to_csv("D:/BugTriage/Chrome_report_Deeptriage.csv", mode='w')

In [36]:
deepChrome = pd.read_csv("D:/BugTriage/Chrome_report_Deeptriage.csv")
deepChrome

,Unnamed: 0,id,issue_id,issue_title,report_time,owner,description
0,0,1,2,Testing if chromium id works,2008-08-30 16:00:21,NaN,testing if chromium id works what steps will r...
1,1,2,3,This is a test,2008-08-31 02:47:11,NaN,this is a test product version : 0.2.149....
2,2,3,4,"Scrolling with some scroll mice (touchpad, etc...",2008-09-02 19:02:31,amit@chromium.org,"scrolling with some scroll mice (touchpad, etc..."
3,3,4,5,Java not working yet,2008-09-02 19:04:27,NaN,java not working yet product version : 0....
4,4,5,6,Chrome treats links in frames differently than...,2008-09-02 19:08:01,NaN,chrome treats links in frames differently than...
...,...,...,...,...,...,...,...
383099,383099,383100,349995,Windows GPU bots failing on multiple tests,2014-03-06 18:42:28,ernstm@chromium.org,windows gpu bots failing on multiple tests all...
383100,383100,383101,349996,Pop-up window from inside Flash Player not wor...,2014-03-06 18:42:58,NaN,pop-up window from inside flash player not wor...
383101,383101,383102,349997,URL blacklist policy can't filter by query par...,2014-03-06 18:43:00,kaliamoorthi@chromium.org,url blacklist policy can't filter by query par...
383102,383102,383103,349998,CSS auto-complete suggestion list is incomplet...,2014-03-06 18:43:04,apavlov@chromium.org,css auto-complete suggestion list is incomplet...


In [ ]:
def augDeep_data(dataset):
    aug_range = len(dataset)
    f = open('Deep_Chrome_aug.csv', 'w', newline = '', encoding='utf-8')
    wr = csv.writer(f)
    for x in range(len(dataset)):
        wr.writerow([dataset['text'][x], dataset['labels'][x]])
    
    for x in range(3):
        for line in range(aug_range):
            augmented_Embeded = aug_word_BackTranslation.augment(dataset['description'][line])
            wr.writerow([augmented_Embeded, dataset['']])